In [1]:
import optuna
from ultralytics import YOLO
import torch
import logging
import numpy as np 
import pandas as pd 


c:\Users\mishr\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())


2.5.0+cu118
True


In [4]:
device

device(type='cuda')

In [5]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [6]:
import optuna
import numpy as np
import logging

logger = logging.getLogger(__name__)

class YOLOTrainer:
    def __init__(self, data_path, model_config, n_trials=20):
        self.data_path = data_path
        self.model_config = model_config
        self.n_trials = n_trials
        self.study = optuna.create_study(
            direction="maximize",
            sampler=optuna.samplers.TPESampler(seed=42),
            pruner=optuna.pruners.MedianPruner(
                n_startup_trials=5,
                n_warmup_steps=5,
                interval_steps=3
            )
        )

    
    def objective(self, trial):
        try:
            # Define hyperparameters using updated Optuna methods
            learning_rate = trial.suggest_float("lr0", 1e-5, 1e-3, log=True)
            momentum = trial.suggest_float("momentum", 0.7, 0.95)
            weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True)
            img_size = trial.suggest_categorical("imgsz", [416, 512, 640])
            mosaic = trial.suggest_float("mosaic", 0.0, 0.8)
            mixup = trial.suggest_float("mixup", 0.0, 0.1)
            hsv_h = trial.suggest_float("hsv_h", 0.0, 0.05)
            hsv_s = trial.suggest_float("hsv_s", 0.0, 0.5)
            hsv_v = trial.suggest_float("hsv_v", 0.0, 0.5)
            degrees = trial.suggest_int("degrees", 0, 30)
            translate = trial.suggest_float("translate", 0.0, 0.1)
            scale = trial.suggest_float("scale", 0.3, 0.7)
            shear = trial.suggest_float("shear", 0.0, 5.0)
            warmup_epochs = trial.suggest_int("warmup_epochs", 3, 5)
            warmup_momentum = trial.suggest_float("warmup_momentum", 0.6, 0.8)
            warmup_bias_lr = trial.suggest_float("warmup_bias_lr", 0.05, 0.1)
            box = trial.suggest_float("box", 6.0, 8.0)
            cls = trial.suggest_float("cls", 0.3, 0.7)
            dfl = trial.suggest_float("dfl", 1.3, 1.7)
            batch_size = trial.suggest_categorical("batch", [8, 16])

            model = YOLO(self.model_config)

            # Training with updated parameters
            results = model.train(
                data=self.data_path,
                epochs=10,
                batch=batch_size,
                imgsz=img_size,
                lr0=learning_rate,
                momentum=momentum,
                weight_decay=weight_decay,
                device='cuda',  # or 'cpu' if no GPU
                amp=False,
                cache=False,
                workers=8,
                verbose=True,
                patience=15,
                mosaic=mosaic,
                mixup=mixup,
                hsv_h=hsv_h,
                hsv_s=hsv_s,
                hsv_v=hsv_v,
                degrees=degrees,
                translate=translate,
                scale=scale,
                shear=shear,
                warmup_epochs=warmup_epochs,
                warmup_momentum=warmup_momentum,
                warmup_bias_lr=warmup_bias_lr,
                box=box,
                cls=cls,
                dfl=dfl,
                overlap_mask=True,
                mask_ratio=4,
                close_mosaic=10,
                label_smoothing=0.0,
                save_period=10,
                exist_ok=True
            )

            # Check for NaN in results
            if np.isnan(results.results_dict.get('metrics/mAP50(B)', 0.0)):
                logger.warning("NaN detected in results!")
                return 0.0

            return results.results_dict.get('metrics/mAP50(B)', 0.0)

        except Exception as e:
            logger.error(f"Error in trial: {str(e)}")
            return 0.0

    
    def optimize(self):
        """Run the optimization process"""
        try:
            self.study.optimize(
                self.objective,
                n_trials=self .n_trials,
            )
            
            # Log best parameters
            logger.info(f"Best parameters: {self.study.best_params}")
            
            # Validate best parameters
            self.validate_best_params()
            
            return self.study.best_params
            
        except Exception as e:
            logger.error(f"Optimization failed: {str(e)}")
            return None

    def validate_best_params(self):
        """Validate the best parameters with a short training run"""
        try:
            model = YOLO(self.model_config)
            
            # Remove any invalid parameters
            valid_params = {k: v for k, v in self.study.best_params.items() 
                          if k not in ['clip']}  # Add other invalid params here if needed
            
            test_results = model.train(
                data=self.data_path,
                epochs=3,  # Short test run
                **valid_params,
                amp=False
            )
            
            if np.isnan(test_results.results_dict.get('metrics/mAP50(B)', 0.0)):
                logger.error("Best parameters produced NaN in validation run!")
                return False
                
            logger.info("Best parameters validated successfully!")
            return True
            
        except Exception as e:
            logger.error(f"Error validating best parameters: {str(e)}")
            return False



In [7]:
trainer = YOLOTrainer(
    data_path=r'D:\All_Coding_stuff\BioTechResearch\640DataSet\Data.yaml',
    model_config='yolov8s.yaml'  
)



[I 2024-10-24 16:05:50,391] A new study created in memory with name: no-name-783615a8-26e8-423c-a910-ee0c66c0929a


In [8]:
best_params = trainer.optimize()

New https://pypi.org/project/ultralytics/8.3.21 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.17  Python-3.11.3 torch-2.5.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.yaml, data=D:\All_Coding_stuff\BioTechResearch\640DataSet\Data.yaml, epochs=10, time=None, patience=15, batch=16, imgsz=416, save=True, save_period=10, cache=False, device=cuda, workers=8, project=None, name=train, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_mask

train: Scanning D:\All_Coding_stuff\BioTechResearch\640DataSet\train\jpg.cache... 4810 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4810/4810 [00:00<?, ?it/s]
val: Scanning D:\All_Coding_stuff\BioTechResearch\640DataSet\valid\jpg.cache... 1195 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1195/1195 [00:00<?, ?it/s]


Plotting labels to runs\detect\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=5.6115164153345e-05' and 'momentum=0.937678576602479' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.000291063591313307), 63 bias(decay=0.0)
Image sizes 416 train, 416 val
Using 8 dataloader workers
Logging results to runs\detect\train
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10      3.02G      2.344      2.322      2.349         16        416: 100%|██████████| 301/301 [01:26<00:00,  3.47it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:11<00:00,  3.37it/s]


                   all       1195       2455      0.399      0.575      0.393      0.113

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       1195       2455      0.777      0.791      0.854      0.421



3 epochs completed in 0.197 hours.
Optimizer stripped from runs\detect\train74\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\train74\weights\best.pt, 22.5MB

Validating runs\detect\train74\weights\best.pt...
WARNING  validating an untrained model YAML will result in 0 mAP.
Ultralytics 8.3.17  Python-3.11.3 torch-2.5.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 6144MiB)
YOLOv8s summary (fused): 168 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 75/75 [00:18<00:00,  3.96it/s]


                   all       1195       2455      0.723      0.774      0.815      0.463
                 Gram+        618       1016      0.665      0.871      0.829      0.477
                 Gram-        856       1439       0.78      0.677      0.802      0.449
Speed: 0.4ms preprocess, 10.4ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs\detect\train74


INFO:__main__:Best parameters validated successfully!


In [9]:
if best_params:
    print("Best hyperparameters found:")
    for key, value in best_params.items():
        print(f"{key}: {value}")
else:
    print("Optimization did not return any parameters.")

Best hyperparameters found:
lr0: 1.0040948800017923e-05
momentum: 0.7703535648947398
weight_decay: 5.209314402340292e-05
imgsz: 640
mosaic: 0.3937779901250769
mixup: 0.029352974531727874
hsv_h: 0.040297644473891385
hsv_s: 0.4080914229305615
hsv_v: 0.32142371529527775
degrees: 2
translate: 0.03653909578724725
scale: 0.34890016696138526
shear: 3.763501888501172
warmup_epochs: 4
warmup_momentum: 0.7969320007259797
warmup_bias_lr: 0.08591454573132772
box: 7.953251039353496
cls: 0.5675368220875538
dfl: 1.3025833598743881
batch: 8
